# Naïve Bayes and Logistic Regression

#### 🎯 Learning Goals

1. Understand the **Naïve Bayes Classifier** and its assumptions, learn to implement it using `sklearn`.
2. Understand the basic idea of **Logistic Regression** and learn to implement it using `sklearn`.

In [1]:
# Load our libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Use a nicer style for plots
plt.style.use("seaborn-v0_8-muted")

___ 
## Data Preprocessing

We will work with data on bank loans to illustrate a use-case of the Naïve Bayes Classifier and the Logistic Regression. 

In [ ]:
# Load data on bank loands and defaults
loans = pd.read_csv("data/loan_defaults.csv")
loans.head()

In [ ]:
# Get a better sense of the data by looking at the summary statistics
loans.describe()

As we can see from the excerpt above, we have a lot of data (195k observations) and there are a lot of different variables (columns) in our dataset. Normally, you aren't just *given* some dataset, but you obtained it from a specific source and you can find some information on what each variable represents or how it is measured. This is not the case here, so you have to rely on the column names&hellip;

Our target variable is the `loan_status`, and a `1` indicates a default, i.e., that the loan was not paid back. As you can already see, without this particular information, it would be hard to understand only from the data whether `0` or `1` means a default.


First, let's check the value of `loan_status`, if there are any missing values, and if it is balanced or not (balanced meaning there is a similar number of occurence for each class).

In [ ]:
# Count of values for the target variable
loans["loan_status"].value_counts()

There are approximately half as many defaults as there are repaid loans, this is expected. The nice thing is that there are no missing values for our label. What about the other columns, do they have missing values?

In [ ]:
# Let's check the percentage of missing values in each column
loans.isnull().sum() / loans.shape[0] * 100

`months_since_last_delinquent` has a whopping 54.71% of missing values, and `current_loan_amount` also has a few, with 18.02%. This raises an issue we have largely ignored until now: how do we deal with missing data?

___
### Handling missing data
Unfortunately, there is no *best* way to handle missing data. As with many aspects of data science, it is more of an art than a science. Here are some possible approaches and what drawback they might entail:

#### 1. Removing columns with missing values
This is what we will do for `months_since_last_delinquent`, more than half of the data is missing so it really is an extreme case. However, when choosing this approach, we could be dropping important variables that might help us for our prediction. This is something we would like to check in general, e.g., if the 45% of non-missing data in this column was able to perfectly predict the loan defaults, we would obviously still prefer keeping it and we would deal with the missing values in a different way.

#### 2. Removing rows with missing values
What if instead of dropping the entire column, we simply dropped every row that has a missing value in any of the column? This also works. However, this can reduce our sample size significantly. For instance, we do not want to drop 55% of our dataset because a single variable is missing, that might impact the analysis too much. Furthermore, what if there is a *reason* why the data is missing? For instance, there might be a *structure* to the missing data and by ignoring the missing data, we might be biasing our model strongly.

For instance, consider that in some geographical area, the data is missing because it was too hard to gather. If we drop missing data, our model will probably perform badly when it comes to predicting observations in that particular area.

#### 3. Imputing missing values
A third method of dealing with missing values is to *impute* them, i.e., replace them with some other value. For example, we might want to replace the `current_loan_amount` values with the mean of the non-missing values. This would allow us to keep our full dataset instead of discarding 18% of it if we were to remove the rows as mentioned above in 2.

The difficulty of this approach is to choose an imputation strategy. Using the mean or the mode of the non-missing values is the simplest way to do it, but it might not be the best. In fact, if there are a lot of missing observations, it might make things worse.

#### 4. Recoding
In our case, a missing value for `months_since_last_delinquent` may mean that the person has never been delinquent on a payment so far. We may create a new variable `ever_delinquent` and set it for all missings to 0 and for non-missings to 1. We could also set the missings to a very high value. Literally speaking, the number of months since last delinquency is infinite (or the age of the person) for those who have never been delinquent. Whether this is a good idea depends also whether you can show to improve the predictive performance of the model with this procedure.

___

As you can take away from this discussion, handling missing data is complex and full of somewhat arbitrary decisions. It is important to double check with our data that our chosen strategy isn't impairing our model, and, most importantly, we must be able to justify our choices.

In this case, learning to deal with missing data is not the main objective of this notebook. So we will take the easy way out: drop `months_since_last_delinquent` from our model and drop all rows that have missing values anywhere else. But be mindful that we could surely obtain a better model by spending more time and effort on our missing data!

In [ ]:
# Drop the months_since_last_delinquent column
loans.drop(columns=["months_since_last_delinquent"], inplace=True)
# Drop the rows with missing values
loans.dropna(inplace=True)
loans.head() # Display the data

In [ ]:
# Let's check the number of occurences for each value in the home_ownership column
loans["home_ownership"].value_counts()

After this cleaning step, we are left with 152k observations.

As you can see, many of the variables have string values. In a statistical model, we need numerical values for all
features (and for the target), so we will need to process these values in some way.

#### Dummy-encoding and One-hot-encoding
The most common way to encode non-numerical variables is the **dummy-encoding** or **one-hot-encoding**. While there is a difference in the encodings, the names are often used interchangeably. As a matter of fact, dummy-encoding is a special case of one-hot-encoding, when there are only two variables. So, one-hot-encoding is just the more general encoding. Generally, the terms are used interchangeably.

Take the variable `term` in our dataframe, it can take values `Short Term` or `Long Term`. This is perfect for dummy-encoding, e.g., we can replace `Short Term` with a `1` and `Long Term` with a `0`. This way, we have replaced the string variable by a single number.

Take `home_ownership` on the other hand. It can take values `Home Mortgage`, `Own Home`, `Rent`, `HaveMortgage` instead, so up to 4 different values, and we won't be able to dummy-encode it. Instead, we can use one-hot-encoding, the idea is to create 3 new variables, also called dummies, (always one less than the possible values, same for dummy-encoding!):  
1. `Home_Mortgage`: takes value `1` if the `Home_Ownership` is equal to `Home Mortgage`, `0` otherwise.
2. `Own_Home`: takes value `1` if the `Home_Ownership` is equal to `Own Home`, `0` otherwise.
3. `Rent`: takes value `1` if the `Home_Ownership` is equal to `Rent`, `0` otherwise.



#### ➡️ ✏️ Task 1

Why do we need one less dummy than the number of possible values a variable can take? Are we not forgetting about `HaveMortgage` or `Long Term`? Can we just get away with discarding information like that? Discuss with your classmates.

In [ ]:
# First let's have a look at how the function works on a single column
# Notice the use of `drop_first=True`, try to see what happens if you set it to false
pd.get_dummies(loans["home_ownership"], drop_first=True)

Unfortunately, the above only returns the encoding for a single column, so we would have to reconstruct our dataframe by replacing every string column with its encoding. This seems cumbersome. Luckily, `pandas` is very flexible, and we can make `get_dummies` work on the full dataframe, transforming only the necessary columns.

In [9]:
# Make a list of columns which have an 'object' type (string)
str_cols = [col for col in loans.columns if loans[col].dtype == "O"]

In [ ]:
# For understanding what the code above does check this
# A type ‘O’ just stands for “object” which in Pandas’ world is a string.
loans["home_ownership"].dtype

In [ ]:
print(str_cols)

# Encode these columns
loans = pd.get_dummies(loans, columns=str_cols, drop_first=True)
loans # Display the data

___

## Modeling

### Naïve Bayes Classifier

Recall how we defined the classification problem. Given some observation $\mathbf{x}$, we want to predict the label $y^{(i)} \in \mathcal{Y}$, where $\mathcal{Y} = \mathcal{C} = \{1, 2, \dots, C\}$ is discrete and potentially unordered.

The Bayes Classifier is the optimal classifier and is defined as follows:

$$
\begin{align*}
    f_\text{Bayes}(\mathbf{x}^{(i)})  &= \underset{c \in \mathcal{C}}{\arg\max} \ P(y^{(i)} =c | X=\mathbf{x}^{(i)}) \\
    &= \underset{c \in \mathcal{C}}{\arg\max} \ \frac{P(X = \mathbf{x}^{(i)} | y^{(i)} = c) \cdot P(y^{(i)}=c)}{P(X=\mathbf{x}^{(i)})}.
\end{align*}
$$

While optimal, the Bayes Classifier is typically intractable, as it requires us to know the joint distribution of all features $X$ and the label $y$.

The Naïve Bayes Classifier is a simplification of the Bayes Classifier, where we assume that all features are independent of each other, given the label. This is a very strong assumption, but it allows us to write the following:

$$
\begin{align*}
    f_\text{Naïve}(\mathbf{x}^{(i)})  &= \underset{c \in \mathcal{C}}{\arg\max} \ P(y^{(i)} =c | X=\mathbf{x}^{(i)}) \\
    &= \underset{c \in \mathcal{C}}{\arg\max} \ \frac{P(X = \mathbf{x}^{(i)} | y^{(i)} = c) \cdot P(y^{(i)}=c)}{P(X=\mathbf{x}^{(i)})} \\ 
    &= \underset{c \in \mathcal{C}}{\arg\max} \ \frac{\prod_{j=1}^p P_c(X_j = x_j^{(i)})}{\sum_{k \in \mathbb{C}}\left(\prod_{l=1}^p P_k(X_l = x_l^{(i)})\right)}.
\end{align*}
$$

Here, $P_c(X_j = x_j^{(i)})$ is the probability that the $j$-th feature takes the value $x_j^{(i)}$, and that the label is $y^{(i)}= c$. This is a much simpler expression, as it only requires us to know the distribution of each feature, given the label. This is much easier to estimate from data.

There are several flavors of Naïve Bayes Classifier, depending on how we choose to estimate the probabilities using our data. We will focus on the simplest variant, called Gaussian Naïve Bayes. We simply assume that all features are normally distributed within each class, and we estimate the mean and variance of each feature, given the label, i.e., $X_j|Y=c\sim \mathcal{N}(\mu_{jc}, \sigma^2_{jc})$. This is a very strong assumption, but it is also very easy to $\mu_{jc}$ and $\sigma^2_{jc}$ from data.

#### ➡️ ✏️ Task 2

Inspect the plots below and the code that generated them to understand what is being plotted (the plots are not very informative on purpose).

Do you think the Gaussian Naïve Bayes Classifier is a good choice for this dataset? Why or why not? Discuss with your classmates. Think of the assumptions we are making and whether they are reasonable.

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(16, 8))

# Columns to plot
cols = ["current_loan_amount", "credit_score", "annual_income", "monthly_debt"]

# In the top row, plot the distributions where loan_status == 0
# In the bottom row, plot the distributions where loan_status == 1
for k in [0, 1]:
    # Get the subset of the data where loan_status == k
    loans_plt = loans[loans["loan_status"] == k]
    for i, ax in enumerate(axs[k, :]):
        # Compute the empirical mean and standard deviation
        mu = loans_plt[cols[i]].mean()
        sigma = loans_plt[cols[i]].std()
        
        # Plot the data (density histogram)
        ax.hist(loans_plt[cols[i]], bins=20, density=True)
        ax.set_title(cols[i])

        # Plot a Gaussian distribution with empirical mean and standard deviation
        x = np.linspace(loans_plt[cols[i]].min(), loans_plt[cols[i]].max(), 100)
        ax.plot(x, 1 / (sigma * np.sqrt(2 * np.pi)) * np.exp(- (x - mu)**2 / (2 * sigma**2)),
                lw=3)

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 6))

# Columns to plot
cols = ["monthly_debt", "maximum_open_credit", "credit_score"]


# In the top row, plot the distributions where loan_status == 0
# In the bottom row, plot the distributions where loan_status == 1
for k in [0, 1]:
    # Get the subset of the data where loan_status == k
    loans_plt = loans[loans["loan_status"] == k]
    for i, ax in enumerate(axs[k, :]):
        ax.scatter(loans_plt[cols[i]], loans_plt[cols[(i + 1) % 3]], alpha=0.3)
        ax.set_xlabel(cols[i])
        ax.set_ylabel(cols[(i + 1) % 3])

In [14]:
# Import Gaussian Naïve Bayes algorithm
from sklearn.naive_bayes import GaussianNB

# Import train_test_split
from sklearn.model_selection import train_test_split

In [15]:
# Prepare features and target
X, y = loans.drop(columns=["loan_status"]), loans["loan_status"]

# Split the data into train and test sets so we can evaluate and compare our models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                            shuffle=True, random_state=72)

In [ ]:
# Create and fit the Gaussian Naïve Bayes classifier
nbc = GaussianNB()

# Fit the model
nbc.fit(X_train, y_train)

In [ ]:
# Compute the number of correct predictions (accuracy) 
# on the training and test sets
print("Train accuracy:", nbc.score(X_train, y_train))
print("Test accuracy:", nbc.score(X_test, y_test))

#### ➡️ ✏️ Task 3

Based on the accuracy, can you assess whether the model is performing well? Let us compare it to another, even more naïve classifier: one that always predicts the most common class.

+ Compute the accuracy of the most common class classifier.
    + Check which class is the most common one.
    + Compute the accuracy if your predictions where always the most common class.
    + Compare the accuracy of the Naïve Bayes Classifier to the accuracy of the most common class classifier on both train and test sets.

*Hint*: You can use the [`accuracy_score`](https://scikit-learn.org/stable/modules/model_evaluation.html#accuracy-score) from `scikit-learn` or also implement it yourself (it is straightforward, we already coded parts of it the `101_ml-intro.ipynb` notebook).

In [18]:
from sklearn.metrics import accuracy_score

In [ ]:
# ➡️ ✏️ Your code here
y_pred_train_mc = np.zeros(len(y_train))
y_pred_test_mc = np.zeros(len(y_test))

y_pred_train_nbc = nbc.predict(X_train)
y_pred_test_nbc = nbc.predict(X_test)

print("Train accuracy of most common:", accuracy_score(y_train, y_pred_train_mc))
print("Test accuracy of most common :", accuracy_score(y_test, y_pred_test_mc))
print("Train accuracy of Naïve Bayes:", accuracy_score(y_train, y_pred_train_nbc))
print("Test accuracy of Naïve Bayes :", accuracy_score(y_test, y_pred_test_nbc))

### Logistic Regression Classifier

Similar to how we defined the problem for Naïve Bayes, we have an observation $\mathbf{x}^{(i)}$ and aim to predict the label $y^{(i)} \in \mathcal{Y}$, where $\mathcal{Y} = \mathcal{C} = \{1, 2, \dots, C\}$. However, for simplicity, let's focus on the binary classification case where $\mathcal{C} = \{0, 1\}$.

Unlike Naïve Bayes, the Logistic Regression Classifier does not model the joint probability of features and labels. Instead, it models the conditional probability directly using the logistic function:

$$
\begin{align*}
    P(y^{(i)} = 1 | X=\mathbf{x}^{(i)}) = \frac{e^{\beta_0 + \beta_1 x_1^{(i)} + \dots + \beta_p x_p^{(i)}}}{1 + e^{\beta_0 + \beta_1 x_1^{(i)} + \dots + \beta_p x_p^{(i)}}}
\end{align*}
$$

Here, $\beta_0, \beta_1, \dots, \beta_p$ are parameters of the model, which we estimate from the data. The classification decision can then be defined as:

$$
\begin{align*}
    f_\text{LR}(\mathbf{x}^{(i)})  &= \underset{c \in \{0,1\}}{\arg\max} \ P(y^{(i)} =c | X=\mathbf{x}^{(i)}) \\
    &= \begin{cases}
    1 & \text{if } P(y^{(i)} = 1 | X=\mathbf{x}^{(i)}) > 0.5,\\
    0 & \text{otherwise}.
    \end{cases}
\end{align*}
$$

In practice, Logistic Regression is used for binary classification, and extensions like Multinomial Logistic Regression (also called Softmax Regression) are used for multi-class problems.

For model training, we maximize the likelihood function, which is equivalent to minimizing the logistic loss (or cross-entropy loss) over all observations:

$$
\begin{align*}
    \mathcal{L}(\beta) &= \sum_{i=1}^N y^{(i)} \log(P(y^{(i)}=1|X=\mathbf{x}^{(i)})) + (1-y^{(i)}) \log(1 - P(y^{(i)}=1|X=\mathbf{x}^{(i)})) \\
    &= \sum_{i=1}^N y^{(i)} \log\left(\frac{e^{\beta_0 + \beta_1 x_1^{(i)} + \dots + \beta_p x_p^{(i)}}}{1 + e^{\beta_0 + \beta_1 x_1^{(i)} + \dots + \beta_p x_p^{(i)}}}\right) + (1-y^{(i)}) \log\left(\frac{1}{1 + e^{\beta_0 + \beta_1 x_1^{(i)} + \dots + \beta_p x_p^{(i)}}}\right).
\end{align*}
$$

#### ➡️ ✏️ Task 4

Repeat the structure implemented above the Naïve Bayes Classifier for the Logistic Regression. Compare the accuracy of the Logistic Regression to the accuracy of the Naïve Bayes Classifier on both train and test sets.

In [ ]:
from sklearn.linear_model import LogisticRegression

lrc = LogisticRegression()
lrc.fit(X_train, y_train)

y_pred_train_lrc = lrc.predict(X_train)
y_pred_test_lrc = lrc.predict(X_test)


print("Train accuracy of most common  :", accuracy_score(y_train, y_pred_train_mc))
print("Test accuracy of most common   :", accuracy_score(y_test, y_pred_test_mc))
print("Train accuracy of Naïve Bayes  :", accuracy_score(y_train, y_pred_train_nbc))
print("Test accuracy of Naïve Bayes   :", accuracy_score(y_test, y_pred_test_nbc))
print("Train accuracy of Logistic Reg.:", accuracy_score(y_train, y_pred_train_lrc))
print("Test accuracy of Logistic Reg. :", accuracy_score(y_test, y_pred_test_lrc))

#### ➡️ ✏️ Task 5

Apply feature transformation of your choice to the features and re-fit both the Naïve Bayes Classifier and the Logistic Regression. Compare the accuracy of the two models on the test set.

In [21]:
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.pipeline import Pipeline

In [22]:
nbc = Pipeline([
    ("feature_transform", PowerTransformer()),
    ("classifier", GaussianNB())
])

lrc = Pipeline([
    ("feature_transform", PowerTransformer()),
    ("classifier", LogisticRegression())
])

In [23]:
nbc.fit(X_train, y_train)
lrc.fit(X_train, y_train)

y_pred_train_nbc = nbc.predict(X_train)
y_pred_train_lrc = lrc.predict(X_train)
y_pred_test_nbc = nbc.predict(X_test)
y_pred_test_lrc = lrc.predict(X_test)

In [ ]:
print("Train accuracy of most common  :", accuracy_score(y_train, y_pred_train_mc))
print("Test accuracy of most common   :", accuracy_score(y_test, y_pred_test_mc))
print("Train accuracy of Naïve Bayes  :", accuracy_score(y_train, y_pred_train_nbc))
print("Test accuracy of Naïve Bayes   :", accuracy_score(y_test, y_pred_test_nbc))
print("Train accuracy of Logistic Reg.:", accuracy_score(y_train, y_pred_train_lrc))
print("Test accuracy of Logistic Reg. :", accuracy_score(y_test, y_pred_test_lrc))

#### ➡️ ✏️ Task 6 (Optional)

Reading the discussion on handling missing data above, think of how you could apply this to the current dataset and observe whether you can improve your models in doing so.

In [29]:
# Reload full data
loans = pd.read_csv("data/loan_defaults.csv")
# Fill missing values with the mean of the column 

# Faster to do it only where we have missing values rather than the following (whole dataframe)
# loans.fillna(loans.mean(), inplace=True)

for col in loans.columns:
    if loans[col].isna().any():
        loans[col] = loans[col].fillna(loans[col].mean())

loans = pd.get_dummies(loans, columns=str_cols, drop_first=True)

In [30]:
# Re-do the train/test split
X, y = loans.drop(columns=["loan_status"]), loans["loan_status"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                            shuffle=True, random_state=72)


In [31]:
# Re-do the fitting
nbc.fit(X_train, y_train)
lrc.fit(X_train, y_train)

y_pred_train_mc = np.zeros(len(y_train))
y_pred_test_mc = np.zeros(len(y_test))
y_pred_train_nbc = nbc.predict(X_train)
y_pred_train_lrc = lrc.predict(X_train)
y_pred_test_nbc = nbc.predict(X_test)
y_pred_test_lrc = lrc.predict(X_test)

In [ ]:
print("Train accuracy of most common  :", accuracy_score(y_train, y_pred_train_mc))
print("Test accuracy of most common   :", accuracy_score(y_test, y_pred_test_mc))
print("Train accuracy of Naïve Bayes  :", accuracy_score(y_train, y_pred_train_nbc))
print("Test accuracy of Naïve Bayes   :", accuracy_score(y_test, y_pred_test_nbc))
print("Train accuracy of Logistic Reg.:", accuracy_score(y_train, y_pred_train_lrc))
print("Test accuracy of Logistic Reg. :", accuracy_score(y_test, y_pred_test_lrc))